# SVD, numerischer Rang und PCA

Gegeben sei eine $10.000\times400$ zentrierte Daten-Matrix $X$ mit Singulärwerten $\sigma_j = 10.000 \cdot \exp{\left(-\sqrt{j+8}\right)}$, $j=1,\ldots,400$.

Berechnen Sie
- $||X||_2$
- den numerischen Rang in Single-Precision (Maschinengenauigkeit $\text{eps} = 1.19209\cdot10^{-7}$)
- den absoluten Fehler einer Best-Approximation von Rang $42$ in der $2$-Norm $||X-X_42||_2$ und wie viel Speicher dabei in Single-Precision eingespart wird (in $\%$)
- wie groß der Rang $r$ einer Best-Approximation gewählt werden muss, damit $||X-X_r||_2 \le 10^{-4} \cdot ||X||_2$
- die totale Varianz $T$
- wie viel $\%$ der Totalen Varianz in den ersten $3$ PCA-Hauptkomponenten liegen.

In [7]:
import numpy as np
np.set_printoptions(precision=4, suppress=True)

n = 10_000
m = 300

def sigma(j):
    return n * np.exp(-1 * np.sqrt(j + 8))

## $2$-Norm

$$
||X||_2 = \max \sigma_j = \sigma_1
$$

In [8]:
sigmas = []

for i in range(1, m + 1):
    sigmas.append(sigma(i))

sigmas = np.array(sigmas)

In [9]:
X2 = np.max(sigmas)
# X2 = sigmas[1 - 1]
print(f'||X||2 = {X2:.4f}')

||X||2 = 497.8707


## Numerischer Rang

Numerischer Rang ($\varepsilon$-Rang) von $\mathbf{X}\in\mathbb{R}^{n\times m}$ mit Singulärwerten $\left( \sigma_j\right)_{j=1}^m$ und Toleranz $0<\varepsilon \ll n:$

$\operatorname{rang}_\varepsilon(\mathbf{X}) = \text{Anzahl Singulärwerte mit}~\sigma_j>\varepsilon$

mit $\varepsilon = \max{(n,m)} \cdot \Vert \mathbf{X} \Vert_2 \cdot \texttt{eps}$

In [10]:
eps = 1.19209 * 1e-7
val = max(n, m) * X2 * eps

rank_eps = np.count_nonzero(sigmas > val)
print(rank_eps)

86


## Absoluter Fehler Best-Approximation

Absoluter Fehler einer Rang-$k$ Best-Approximation: $||\mathbf{X} - \mathbf{X}_k||_2 = \sigma_{k+1}$

In [12]:
k = 42
round(sigma(k + 1), 4)

7.9162

### Speichereinsparung

$\text{Speicher}=\#\text{Zeilen} \cdot \# \text{Spalten} \cdot 32~\text{Bit}$

$\text{Speicher}_r = nk + k + mk~(U_k \Sigma_k V^\intercal_r: n\times k + k + m \times k)$

In [36]:
speicher_original = n * m * 32
# speicher_rang_approx = (n*k + k + m*k) * 32
speicher_rang_approx = k * (n + m + 1) * 32

einsparung = (1 - speicher_rang_approx / speicher_original) * 100
print(f'{einsparung:.4f}%')

85.5786%


In [29]:
k * (n + m + 1)

432642

In [30]:
n*k + k*k + m*k

434364

## Bestimmung Rang

> Wie groß der Rang $r$ einer Best-Approximation gewählt werden muss, damit $||X-X_r||_2 \le 10^{-4} \cdot ||X||_2$?

$$
\begin{aligned}
\Vert X - X_r \Vert_2 &\le 10^{-4} \cdot \Vert X \Vert_2 \\
\Rightarrow \sigma_{r+1} &\le 10^{-4} \cdot \sigma_1
\end{aligned}
$$

In [20]:
goal = 10**-4 * X2
print(f'goal = {goal:.4f}')

for i in range(1, m + 1):
    lhs = sigma(i + 1)
    if lhs <= goal:
        print(f'r = {i} mit lhs = {lhs:.4f}')
        break

goal = 0.0498
r = 141 mit lhs = 0.0480


## Totale Varianz

$$
T = \frac{1}{n - 1} \Vert \hat{X} \Vert_F^2 = \frac{1}{n - 1} \sum_{j=1}^\ell \sigma_j^2
$$

In [23]:
sigmas = np.array(sigmas)
XF = np.sum(sigmas ** 2)
T = 1 / (n - 1) * XF
print(f'T = {T:.4f}')

T = 99.8466


## Prozent in PCA

> Wie viel $\%$ der Totalen Varianz liegen in den ersten $3$ PCA-Hauptkomponenten?

$$
\frac{\frac{1}{n - 1} \sum_{j=1}^{{\color{red} q} \le \ell} \sigma_j^2}{T} 
= \frac{\sum_{j=1}^{{\color{red} q}}}{\sum_{j=1}^{\ell} \sigma_j^2}
\le \operatorname{Tol} < 1
$$

In [28]:
q = 3

Tp = 1 / (n - 1) * np.sum(sigmas[:q] ** 2) / T * 100
print(f'{Tp:.4f}%')

55.9554%
